# Open Data with CMS - outreach and education

### What is Open Data and why is it important?

<img src = "https://yt3.ggpht.com/a-/ACSszfENmQ2SCS7AnguI8q_ZZlNl2Ne_CC0n2lA_xA=s900-mo-c-c0xffffffff-rj-k-no" align = "right"  style = "height:200px" >

CMS produces a huge amount of data and releases it publicly in a manner very few other experiments at the top of their relative fields do. On one hand that data is very easily available to the general public, on the other it is a bit cryptic to use without someone experienced at hand to explain a few things. Here we aim to present some tools to make it very easy and fast to use.

Obviously the data is very important to the scientists or we wouldn't be doing this experiment. It is also very valuable that it can be shared with others: it can make it easier for non-scientists to understand what your work is about, it can give educators interesting tools that promote modern research, it can hook a curious student to get into sciences and so forth. Especially in education there is this problem of "relevance" and "authenticity", ie. how do things done in school relate to the students' lives or to what is actually happening in scientific research today, and using actual data from actual research (from experiments obviously unperformable in classrooms) might prove to be an important asset in closing that gap. 

### So how do we do it?

<img src = "https://cdn-images-1.medium.com/max/346/0*I3hkRieQ6B3qwwhy." align = "right" style = "height:200px">

There are a few ways. You could use event visualizers, spreadsheet programs, particle physics masterclass materials and what not, all of which can be found through [the Open Data portal](http://opendata.cern.ch/), but the one we'll focus on here is Jupyter. You are currently reading a Jupyter notebook, an interactive file that can trivially incorporate text, pictures, animations and most importantly, code. It can support many languages, but we'll use Python 3 here as it is intuitive enough to be used even with people who have absolutely no programming background.

Best thing is that you can add to it on the fly and let people explore the data as they want without them needing to install anything. Especially in education: people hate it when they have to install things and learn to use a bunch of programs they might use every now and then (or just once), teenagers even more so. With Jupyter you don't have to: we can use [MyBinder](www.mybinder.org) to create a virtual workspace and get to work in a minute or two without any hassle, completely browser-based. There are also others, like Google's [colab](colab.research.google.com), if it suits your preferences better.  

CERN's own [SWAN](https://swan.web.cern.ch/) (Service for Web based ANalysis) is also be based on Jupyter, though it requires you to have CERNbox.


### Right, let's get some examples on the table.

As you can see (and test by double clicking around), this document is made of *cells*. Those can have either text (in markdown format) or code in them. To run a cell, select it (so it gets green) and press *ctrl + Enter*. On the left side of the code cells there's a [ ]. If it is empty, the cell has not been run. If it shows an asterisk, the cell is running. If there is a number, it has been run.

If the system somehow gets stuck, just select *Kernel $\rightarrow$ Restart & Clear Output* from the upper menus. This clears any weird leftovers or hangups that might be left in the kernel. If otherwise unsure of commands, press **h** when no cells are chosen to bring up key bindings.


In [ ]:
# This notebook runs on Python 3, so we need to import some modules.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation
from scipy.stats import norm


%matplotlib inline

When that is done, let's get some data in here. We'll use the 2012 sample and a dataset comprising of events where exactly two muons were detected without worrying about any trigger selections (using code similar to the one used in [record 552](http://opendata.cern.ch/record/552)).

This is an example of a simplified derived dataset, but you could very well use any set you want to make from the openly published results. All derived sets can be found from [here](
http://opendata.cern.ch/search?page=1&size=20&q=&experiment=CMS&type=Dataset&subtype=Derived).

In [ ]:
muons = pd.read_csv('http://opendata.cern.ch/record/545/files/Dimuon_DoubleMu.csv')

Getting a picture of "what does particle physics data actually look like" can be a bit hard for those who don't practise it. It's easy to show them, just run the cell below.

In [ ]:
# This shows top 5 rows of data, though you can put a desired number in the parentheses.

muons.head()

Okay, seems legit. That is still a bit dense if left up to the reader, but with a bit of helpful explaining or physics background it starts to make sense. Importantly we can show that there is quite a lot of it, which means it's going to require computers and programming to analyze it.

In [ ]:
# This shows the number of rows in the data.

len(muons)

Though that number seems small to people who are used to these experiments, it is still a huge number for others. It also doesn't tell us anything, whereas a good graphical visualization does. So let's plot a histogram.

In [ ]:
fig = plt.figure(figsize=(15, 10))

plt.hist(muons.M, 300, range = (0,150))

plt.xlabel('Invariant mass (GeV/$c^2$)', fontsize = 15)
plt.ylabel('Number of events \n', fontsize = 15)
plt.title('Measured distribution of events with two muons \n', fontsize = 15)

plt.show()

Now that we see how the data plots out, we can focus on using it to show different things to the intended audience. Say you want to show them the bump at 90 GeV, for an example. Let's extract those results that fall within 80-100 GeV.

In [ ]:
bump = muons[(muons.M >= 80) & (muons.M <= 100)]

fig = plt.figure(figsize=(15, 10))

plt.hist(bump.M, 300, range = (80,100))

plt.xlabel('Invariant mass (GeV/$c^2$)', fontsize = 15)
plt.ylabel('Number of events \n', fontsize = 15)
plt.title('Measured distribution of events with two muons \n', fontsize = 15)

plt.show()

Let's follow with a question "how do these correlate with energies the measured particles have"? You can set the desired threshold for high $p_t$ in the code below and see how it changes the plot. At which point does the low part stop to contribute?

In [ ]:
threshold = 30

highE = bump[(bump.pt1 >= threshold) & (bump.pt2 >= threshold)]
lowE = bump[(bump.pt1 < threshold) & (bump.pt2 < threshold)]

fig = plt.figure(figsize=(15, 10))

plt.hist(highE.M, 300, range = (80,100), alpha = 0.5 , color = 'black', label = 'High energy')
plt.hist(lowE.M, 300, range = (80,100), alpha = 0.5 , color = 'red', label = 'Low energy')

plt.xlabel('Invariant mass (GeV/$c^2$)', fontsize = 15)
plt.ylabel('Number of events \n', fontsize = 15)
plt.title('Measured distribution of events with two muons \n', fontsize = 15)
plt.legend()

plt.show()

We could also try to find out how the measurements are spread spatially.

In [ ]:
prapMuons = pd.concat([muons['eta1'],muons['eta2']])
degMuons = prapMuons.copy()
degMuons[:]=[2*np.arctan(np.exp(x))*360/(2*np.pi) for x in degMuons]

In [ ]:
plt.hist(prapMuons, bins=100, range=(-3,3))

plt.xlabel('Pseudorapidity', fontsize=15)
plt.ylabel('Number of events', fontsize=15)
plt.title('Spatial distribution \n', fontsize=15)

plt.show()

plt.hist(degMuons, bins=180, range=(0,180))

plt.xlabel('Degrees', fontsize=15)
plt.ylabel('Number of events', fontsize=15)
plt.title('Spatial distribution \n', fontsize=15)

plt.show()

There is certainly a dip in the middle. Could it have something to do with the $p_t$?

In [ ]:
transverse = pd.concat([muons.pt1,muons.pt2])
plt.scatter(prapMuons, transverse, s=0.5)

plt.title('$p_t$ in regard to $\eta$ \n', fontsize=15)
plt.xlabel('Pseudorapidity', fontsize=15)
plt.ylabel('Transverse momentum (GeV)', fontsize=15)

plt.show()

plt.scatter(prapMuons, transverse, s=0.5)

plt.ylim(0,6)

plt.title('$p_t$ in regard to $\eta$ \n', fontsize=15)
plt.xlabel('Pseudorapidity', fontsize=15)
plt.ylabel('Transverse momentum (GeV)', fontsize=15)

plt.show()

We could also use pseudorapidity to see how accurately we can measure things dependant on the angle.

In [ ]:
high = 1.5
low = 0.88

# Check the whole data.

highEta = muons[(abs(muons.eta1) >= high) & (abs(muons.eta2) >= high)]
lowEta = muons[(abs(muons.eta1) < low) & (abs(muons.eta2) < low)]

# Or uncomment and run this to check the 90 GeV bump (which was defined a bit above).

# highEta = bump[(abs(bump.eta1) >= high) & (abs(bump.eta2) >= high)]
# lowEta = bump[(abs(bump.eta1) < low) & (abs(bump.eta2) < low)]

In [ ]:
fig = plt.figure(figsize=(10, 8))

h = len(highEta)
l = len(lowEta)

plt.hist(highEta.M, bins=300, range=(0,100), alpha=0.5, label='High $\eta$, n = %i' %h, facecolor='black')
plt.hist(lowEta.M, bins=300, range=(0,100), alpha=0.5, label='Low $\eta$, n = %i' %l)

# Adjust range and ylim accordingly, if you want to focus on the bump.

plt.ylim(0,1500)

plt.xlabel('Invariant mass [GeV/c²]', fontsize=15)
plt.ylabel('Number of events', fontsize=15)
plt.title('Detected muons in regard to $\eta$ \n', fontsize=15)
plt.legend (loc='upper right', fontsize=15)

plt.show()

While more proper and complex curve fits take a bit more effort, you can also pretty easily show some rough distributions on the fly. For a more rigorous show you could use something like ROOT, but that can get a bit burdensome for some situations. Let's take a look at the J/$\psi$ peak with nimble Python.

In [ ]:
fig = plt.figure(figsize=(15, 10))

lower = 2.8
upper = 3.4

fit_i = 3.05
fit_f = 3.15

jpsi = muons[(muons.M >= lower) & (muons.M <= upper)]
fit = muons[(muons.M >= fit_i) & (muons.M <= fit_f)]

coefficient = len(fit)/len(jpsi)
(mu, sigma) = norm.fit(fit.M)

n, bins, patches = plt.hist(jpsi.M, 300, density = 1, facecolor = 'b', alpha=0.5, histtype='stepfilled',
                            range=(lower, upper))

y = coefficient*norm.pdf( bins, mu, sigma)
l = plt.plot(bins, y, 'k--', linewidth = 6)

plt.xlabel('Invariant mass (GeV/$c^2$)', fontsize = 15)
plt.ylabel('Probability \n', fontsize = 15)
plt.title( 'J/$\psi$ normed to one   $\mu$ = %f , $\sigma$ = %f \n' %(mu,sigma), fontsize=15)

cur_axes = plt.gca()
cur_axes.axes.get_yaxis().set_ticks([])

plt.show()


You could also use animations, although those require a bit more dedication from your audience as they are quite slow to create when more than a few frames are used. Like *really* slow, the animation down below plots 25 000 datapoints and took about 8 minutes to make on a low-grade laptop.

In [ ]:
data = muons['M'].copy()

def updt_hist(num, data):
    plt.cla() # this clears previous axes
    axes = plt.gca() # this gets current axes
    # axes.set_ylim(0,1500)
    # axes.set_xlim(0,125)
    plt.hist(data[:num*50], bins = 200) # here we can define the amount of events per step

In [ ]:
%%capture
# this magic function prevents unsightly extra figures popping to view
fig = plt.figure(figsize=(15,10))

anim = matplotlib.animation.FuncAnimation(fig, updt_hist, frames = 500, fargs = (data,))

from IPython.display import HTML
HTML(anim.to_jshtml())

In [ ]:
HTML(anim.to_jshtml())